In [5]:
import os
import langchain



In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

answer_model

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [12]:

llm = OpenAI(model_name = "text-ada-001", openai_api_key = openai_api_key)
chat = ChatOpenAI(temperature=.9, openai_api_key=openai_api_key)

In [13]:
my_text = "what day comes after Friday?"

In [14]:
llm_answer = llm(my_text)
print(llm_answer)



Saturday.


In [15]:
ai_answer = chat(
    [
        SystemMessage(content="You are nice Ai Bot that helps user question"),
        HumanMessage(content=my_text)

    ]
)

In [16]:
ai_answer

AIMessage(content='The day that comes after Friday is Saturday.', additional_kwargs={})

In [21]:
from langchain.schema import Document

In [22]:
my_document = Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })

In [24]:
print(my_document)

page_content="This is my document. It is full of text that I've gathered from other places" metadata={'my_document_id': 234234, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019}


Embadding model

In [25]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [26]:

text = "你好，我不喜歡你"

In [27]:
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None)).


Your embedding is length 1536
Here's a sample: [0.00025405424125446403, -0.00605601788895559, -0.0023420623737650335, -0.029368669672014815, -0.043621112964367374]...


In [29]:
text = "你好，我不喜歡你"
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 1536
Here's a sample: [-0.029608574827122848, -0.013502116881534276, 0.011498291666042285, -0.009475502860798407, -0.04348996536633992]...


In [30]:
text = "同時，加碼祭出普發6,000元優惠大禮包，將集結台新ATM、Richart數位銀行、行動銀行、Richart Life生態圈及台新Pay，搭配推出眾多好康回饋。鼓勵選擇「登記入帳」的民眾，趁此機會開立Richart帳戶，完成指定任務，最高可享1,000元現金回饋（含新開戶回饋300元、Richart Mastercard 悠遊金融卡任刷一筆回饋200元及申購指定外幣專案回饋500元），直接將政府給的6,000元放大成7,000元。"
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None)).


Your embedding is length 1536
Here's a sample: [-0.02586610677912013, -0.01716800634427833, 0.007979968756030534, -0.035785698267449685, -0.04061797359631874]...


In [31]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None)).


Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
LLM Output: 
Visit the iconic Colosseum, take a stroll through the breathtaking Trevi Fountain, and explore the ancient ruins of the Roman Forum.


input example 後使 GPT 依 EXAMPLE 回答

In [32]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [35]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(openai_api_key=openai_api_key), 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=2
    )

In [39]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)


In [40]:
# Select a noun!
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None)).


Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [41]:

llm(similar_prompt.format(noun=my_noun))

' classroom'

In [42]:

from langchain.document_loaders import HNLoader

In [43]:
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

In [44]:
data = loader.load()

In [45]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman 3 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very Ozzie_osman 3 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index)
